In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
#read input files
date_map = pd.read_csv('C:/Users/Anurag.Gandhi/Desktop/date_map.csv')
promotions = pd.read_csv('C:/Users/Anurag.Gandhi/Desktop/hnp_promotions.csv')
#read transaction data
txn_data = pd.read_csv('C:/Users/Anurag.Gandhi/Desktop/txn_data_2_years.csv'
                  ,encoding = "ISO-8859-1")

In [5]:
txn_data['NEW_DATE'] = pd.to_datetime(txn_data['TRANSACTION_DATE'], format = '%d%b%Y')
date_map['NEW_DATE'] = pd.to_datetime(date_map['CALENDAR_DATE'], format = '%m/%d/%Y')

In [6]:
txn_data = pd.merge(txn_data, date_map, how = 'left', on = 'NEW_DATE', sort = False)


In [7]:
txn_data.rename(columns = {'WEEK_NO': 'PROM_WEEK_NO'}, inplace = True)
promotions.rename(columns = {'sku': 'SKU', 'prom_week_no': 'PROM_WEEK_NO'}, inplace = True)

In [8]:
txn_data = pd.merge(txn_data, promotions[promotions['zone_id'] == 5][['SKU', 'PROM_WEEK_NO', 'promotions', 'promotions_NC']], 
                 how = 'left', on = ['SKU', 'PROM_WEEK_NO'], sort = False)

In [9]:
txn_data[['promotions','promotions_NC']] = txn_data[['promotions','promotions_NC']].fillna(0)


In [10]:
#Unique ID for Customer-Day (proxy for transaction)
txn_data['UNIQUE_CUST_TRANS'] = txn_data['INFERRED_CUSTOMER_ID'].map(str) + txn_data['TRANSACTION_DATE']

In [11]:
#read correlation output
sisters_list = pd.read_csv('H:/Depository/Sirsa/Promotion_Analytics/Research/Anurag/Basket/sisters_corr_hnp_v3.csv',encoding = "ISO-8859-1")

#business inputs 
#sisters_list = pd.DataFrame(sisters_list[sisters_list['buyer_id_2'].isin([4494,4266,4502])]).reset_index

OSError: File b'H:/Depository/Sirsa/Promotion_Analytics/Research/Anurag/Basket/sisters_corr_hnp_v3.csv' does not exist

In [ ]:
sisters_list_corr = sisters_list

    

## Lift (X->Y) = confidence(X->Y) / support(Y)


In [24]:
trans_sku = txn_data[['UNIQUE_CUST_TRANS','SKU','PROM_WEEK_NO']].drop_duplicates()
#loop for each combination
start_time = time.time()
for i in range(0,len(sisters_list)):
#for i in range(0,363):
    #SKU A: Dishwash SKU
    SKU_A = sisters_list.ix[i,'SKU_1']
    
    #SKU B: Sister SKU
    SKU_B = sisters_list.ix[i,'SKU_2']
    
    #filter weeks where only ONE of the products was on promotion
    test = txn_data[txn_data['SKU'].isin([SKU_A,SKU_B])][['PROM_WEEK_NO', 'promotions', 'SKU']].drop_duplicates()
    test_2 = test.pivot(index = 'PROM_WEEK_NO', columns = 'SKU', values = 'promotions').fillna(0)
    test_2['sum'] = test_2[SKU_A] + test_2[SKU_B]
    test_2['product'] = test_2[SKU_A] * test_2[SKU_B]
    prom_week_list = test_2[(test_2['sum'] > 0) & (test_2['product'] == 0)].index.values
    
    #Customers who bought these two SKUs:
    hnp_pair = txn_data[txn_data['SKU'].isin([SKU_A,SKU_B])][['INFERRED_CUSTOMER_ID', 'SKU']].drop_duplicates()
    sku_count_by_customer = hnp_pair.groupby( ['INFERRED_CUSTOMER_ID'], sort = False).count()
    
    #ATLEAST 1 customer should have bought both the products (this removes unrelated products)
    if ((max(sku_count_by_customer['SKU']) > 1) & (len(prom_week_list) > 0 )):
        
        #distinct SKU transactions
        trans_sku_pair = trans_sku[trans_sku['PROM_WEEK_NO'].isin(prom_week_list)][['UNIQUE_CUST_TRANS','SKU']]
        trans_sku_pair = trans_sku_pair[trans_sku_pair['SKU'].isin([SKU_A,SKU_B])].drop_duplicates()
        pivoted = trans_sku_pair.groupby( ['UNIQUE_CUST_TRANS'], sort = False).count()
        
        #Support (A) = number/fraction of transactions with A
        support_a = trans_sku_pair[trans_sku_pair['SKU'] == SKU_A].shape[0]
        
        #Suport (B) = number/fraction of transactions with B
        support_b = trans_sku_pair[trans_sku_pair['SKU'] == SKU_B ].shape[0]
        
        #Support (A, B) = number/fraction of transactions with both A & B
        intersection = pivoted[pivoted['SKU'] == 2].shape[0]
        sisters_list.ix[i,'SUPPORT A'] = support_a
        sisters_list.ix[i,'SUPPORT B'] = support_b
        sisters_list.ix[i,'SUPPORT A_B'] = intersection
        
        #Confidence = Support (A, B) / Support (A)
        sisters_list.ix[i,'CONFIDENCE'] = np.float64(intersection)/support_a
        
        #Lift = Support (A, B) / ( Support (A) * Support (B) )
        sisters_list.ix[i,'LIFT'] = np.float64(intersection)/(support_a*support_b)
    else:
        sisters_list.ix[i,'LIFT'] = 1000
    print (i + 1, " of", len(sisters_list), " done.")
end_time = time.time()
print ("Elapsed Time:",(end_time - start_time))

1  of 306  done.
2  of 306  done.
3  of 306  done.
4  of 306  done.
5  of 306  done.
6  of 306  done.
7  of 306  done.
8  of 306  done.
9  of 306  done.
10  of 306  done.
11  of 306  done.
12  of 306  done.
13  of 306  done.
14  of 306  done.
15  of 306  done.
16  of 306  done.
17  of 306  done.
18  of 306  done.
19  of 306  done.
20  of 306  done.
21  of 306  done.
22  of 306  done.
23  of 306  done.
24  of 306  done.
25  of 306  done.
26  of 306  done.
27  of 306  done.
28  of 306  done.
29  of 306  done.
30  of 306  done.
31  of 306  done.
32  of 306  done.
33  of 306  done.
34  of 306  done.
35  of 306  done.
36  of 306  done.
37  of 306  done.
38  of 306  done.
39  of 306  done.
40  of 306  done.
41  of 306  done.
42  of 306  done.
43  of 306  done.
44  of 306  done.
45  of 306  done.
46  of 306  done.
47  of 306  done.
48  of 306  done.
49  of 306  done.
50  of 306  done.
51  of 306  done.
52  of 306  done.
53  of 306  done.
54  of 306  done.
55  of 306  done.
56  of 306  done.
5

In [25]:
sisters_list

,buyer_id_2,buyer_resp_2,sub_category_2,sub_category_name_2,segment_2,segment_name_2,sku_desc_2,buyer_id_1,buyer_resp_1,sub_category_1,...,SKU_1,SKU_2,count_stores,avg_corr,base,SUPPORT A,SUPPORT B,SUPPORT A_B,CONFIDENCE,LIFT
0,4502,Buyer Dishwash,389,Dishwash,4744,Other Brands,Ecover dishwasher tablets x25,4502,Buyer Dishwash,389,...,7680076,2947365,7,-0.362695,30,14848,4489,0,0.000000,0.000000e+00
1,4502,Buyer Dishwash,389,Dishwash,4744,Other Brands,Ecover dishwasher tablets x25,4502,Buyer Dishwash,389,...,7680058,2947365,13,-0.383280,30,4230,6222,0,0.000000,0.000000e+00
2,4502,Buyer Dishwash,389,Dishwash,4744,Other Brands,Ecover dishwasher tablets x25,4502,Buyer Dishwash,389,...,7734368,2947365,20,-0.426200,30,4311,5648,0,0.000000,0.000000e+00
3,4502,Buyer Dishwash,389,Dishwash,4744,Other Brands,Ecover dishwasher tablets x25,4502,Buyer Dishwash,389,...,7512416,2947365,13,-0.434300,30,13287,5548,0,0.000000,0.000000e+00
4,4502,Buyer Dishwash,389,Dishwash,4744,Other Brands,Ecover dishwasher tablets x25,4502,Buyer Dishwash,389,...,7513133,2947365,20,-0.377374,30,7829,5548,0,0.000000,0.000000e+00
5,4502,Buyer Dishwash,389,Dishwash,4744,Other Brands,Ecover dishwasher tablets x25,4502,Buyer Dishwash,389,...,7512242,2947365,13,-0.417984,30,9239,5326,0,0.000000,0.000000e+00
6,4502,Buyer Dishwash,389,Dishwash,4744,Other Brands,Ecover dishwasher tablets x25,4502,Buyer Dishwash,389,...,7614246,2947365,6,-0.450226,30,844,4467,0,0.000000,0.000000e+00
7,4502,Buyer Dishwash,389,Dishwash,4744,Other Brands,Ecover dishwasher tablets x25,4502,Buyer Dishwash,389,...,7680127,2947365,7,-0.357809,30,10225,4489,0,0.000000,0.000000e+00
8,4502,Buyer Dishwash,389,Dishwash,4744,Other Brands,Ecover dishwasher tablets x25,4502,Buyer Dishwash,389,...,7512240,2947365,15,-0.394791,30,11258,5326,0,0.000000,0.000000e+00
9,4502,Buyer Dishwash,389,Dishwash,4898,Finish Tablets,Finish All in 1 x65,4502,Buyer Dishwash,389,...,7680093,6718255,17,-0.382710,30,6361,735,0,0.000000,0.000000e+00


In [64]:
sisters_list[sisters_list['sub_category_2'] == 417]

,index,buyer_id_2,buyer_resp_2,sub_category_2,sub_category_name_2,segment_2,segment_name_2,sku_desc_2,buyer_id_1,buyer_resp_1,...,SKU_1,SKU_2,count_stores,avg_corr,avg_corr_np,LIFT,SUPPORT A,SUPPORT B,SUPPORT A_B,CONFIDENCE
76,356,4362,Buyer Pizza & Chilled Bread,417,Chilled Pizza,8768,Thin & Crispy Pizzas,"JS Thin & Crispy cheese & tom Pizza 10""",4362,Buyer Pizza & Chilled Bread,...,7714806,1386134,8,-0.424138,-0.146312,1.000000e+03,NaN,NaN,NaN,NaN
77,362,4362,Buyer Pizza & Chilled Bread,417,Chilled Pizza,8768,Thin & Crispy Pizzas,"JS Thin & Crispy ham&pineapple Pizza 10""",4362,Buyer Pizza & Chilled Bread,...,7714646,1386141,36,-0.413091,-0.396754,1.000000e+03,NaN,NaN,NaN,NaN
78,363,4362,Buyer Pizza & Chilled Bread,417,Chilled Pizza,8768,Thin & Crispy Pizzas,"JS Thin & Crispy ham&pineapple Pizza 10""",4362,Buyer Pizza & Chilled Bread,...,7714684,1386141,38,-0.424861,-0.432439,1.000000e+03,NaN,NaN,NaN,NaN
79,364,4362,Buyer Pizza & Chilled Bread,417,Chilled Pizza,8768,Thin & Crispy Pizzas,"JS Thin & Crispy ham&pineapple Pizza 10""",4362,Buyer Pizza & Chilled Bread,...,7714717,1386141,38,-0.447021,-0.474186,1.000000e+03,NaN,NaN,NaN,NaN
80,365,4362,Buyer Pizza & Chilled Bread,417,Chilled Pizza,8768,Thin & Crispy Pizzas,"JS Thin & Crispy ham&pineapple Pizza 10""",4362,Buyer Pizza & Chilled Bread,...,7714760,1386141,26,-0.402691,-0.400519,1.000000e+03,NaN,NaN,NaN,NaN
81,366,4362,Buyer Pizza & Chilled Bread,417,Chilled Pizza,8768,Thin & Crispy Pizzas,"JS Thin & Crispy ham&pineapple Pizza 10""",4362,Buyer Pizza & Chilled Bread,...,7714705,1386141,31,-0.397856,-0.368772,1.000000e+03,NaN,NaN,NaN,NaN
82,367,4362,Buyer Pizza & Chilled Bread,417,Chilled Pizza,8768,Thin & Crispy Pizzas,"JS Thin & Crispy ham&pineapple Pizza 10""",4362,Buyer Pizza & Chilled Bread,...,7714806,1386141,10,-0.492644,-0.284154,1.000000e+03,NaN,NaN,NaN,NaN
101,495,4362,Buyer Pizza & Chilled Bread,417,Chilled Pizza,548,Pizza Express,PZE classic margherita 245g,4362,Buyer Pizza & Chilled Bread,...,7714760,3659274,40,-0.649258,-0.256820,2.024634e-07,8759,81201,144,0.016440
102,496,4362,Buyer Pizza & Chilled Bread,417,Chilled Pizza,548,Pizza Express,PZE classic margherita 245g,4362,Buyer Pizza & Chilled Bread,...,7714684,3659274,40,-0.734240,-0.263528,9.000609e-08,17240,81201,126,0.007309
103,497,4362,Buyer Pizza & Chilled Bread,417,Chilled Pizza,548,Pizza Express,PZE classic margherita 245g,4362,Buyer Pizza & Chilled Bread,...,7714806,3659274,12,-0.688674,-0.227097,1.078303e-07,2741,81201,24,0.008756


In [26]:
results = sisters_list[['SKU_1','sku_desc_1', 'sub_category_name_1', 'SKU_2', 'sku_desc_2', 'sub_category_name_2', 
                        'SUPPORT A', 'SUPPORT B', 'SUPPORT A_B', 'CONFIDENCE', 'LIFT', 'count_stores', 'base', 'avg_corr']]

In [20]:
sisters_list.columns

Index(['buyer_id_2', 'buyer_resp_2', 'sub_category_2', 'sub_category_name_2',
       'segment_2', 'segment_name_2', 'sku_desc_2', 'buyer_id_1',
       'buyer_resp_1', 'sub_category_1', 'sub_category_name_1', 'segment_1',
       'segment_name_1', 'sku_desc_1', 'SKU_1', 'SKU_2', 'count_stores',
       'avg_corr', 'base', 'SUPPORT A', 'SUPPORT B', 'SUPPORT A_B',
       'CONFIDENCE', 'LIFT'],
      dtype='object')

In [21]:
results[(results['LIFT'] < 3e-7) ]

,SKU_1,sku_desc_1,sub_category_name_1,SKU_2,sku_desc_2,sub_category_name_2,SUPPORT A,SUPPORT B,SUPPORT A_B,CONFIDENCE,LIFT,count_stores,base,avg_corr
0,7693788,FRijj Supreme White Choc & Raspbery375ml,Flavoured Milk,181358,JS British Whole Milk 1 Pint 568ml,Own Label Milk,1105,21746,8,0.007240,3.329265e-07,9,35,-0.427984
7,7697653,Starbucks Skinny Latte 220ml,Flavoured Milk,425490,JS British Organic Whole Mlk 1Pint 568ml,Own Label Milk,418,7784,0,0.000000,0.000000e+00,1,5,-0.342976
8,7727603,Vita Coco Cafe Latte 330ml,Flavoured Milk,425490,JS British Organic Whole Mlk 1Pint 568ml,Own Label Milk,62,951,0,0.000000,0.000000e+00,7,34,-0.393231
10,7353344,JS W.Country Semi Skim Milk 4Pint 2.272L,Own Label Milk,425605,JS British Whole Milk 6 Pint 3.408L,Own Label Milk,13816,6950,0,0.000000,0.000000e+00,1,2,-0.413378
11,7697653,Starbucks Skinny Latte 220ml,Flavoured Milk,464390,JS British Semi Skim Milk 6 Pint 3.408L,Own Label Milk,418,103025,2,0.004785,4.644202e-08,1,3,-0.323789
14,7697651,Starbucks Skinny Cappuccino 220ml,Flavoured Milk,1137637,JS British Semi Skim Milk 2 Pint 1.136L,Own Label Milk,1255,181825,79,0.062948,3.462022e-07,7,28,-0.522640
18,7697653,Starbucks Skinny Latte 220ml,Flavoured Milk,1137729,JS British Skimmed Milk 2 Pint 1.136L,Own Label Milk,418,105117,16,0.038278,3.641420e-07,1,3,-0.392946
19,7697653,Starbucks Skinny Latte 220ml,Flavoured Milk,1186246,JS British Organic Skim Milk 1Pint 568ml,Own Label Milk,418,7831,0,0.000000,0.000000e+00,2,5,-0.435503
21,7697651,Starbucks Skinny Cappuccino 220ml,Flavoured Milk,1208443,JS British Organic Whl Mlk 4Pint 2.272L,Own Label Milk,1151,21986,0,0.000000,0.000000e+00,8,33,-0.465790
22,7727603,Vita Coco Cafe Latte 330ml,Flavoured Milk,1208443,JS British Organic Whl Mlk 4Pint 2.272L,Own Label Milk,170,30108,0,0.000000,0.000000e+00,7,34,-0.408715


In [27]:
results_0_lift = results[ (results['LIFT'] < 1e-6) & (results['SUPPORT A'] >= 300) & (results['SUPPORT B'] >=300) ]
results_0_lift['SKU_1'].drop_duplicates().shape

(31,)

In [29]:
results_0_lift.to_csv('results_dishwash_3.csv')

In [21]:
results_0_lift

,SKU_1,sku_desc_1,sub_category_name_1,SKU_2,sku_desc_2,sub_category_name_2,SUPPORT A,SUPPORT B,SUPPORT A_B,CONFIDENCE,LIFT,count_stores,base,avg_corr
2,7614252,Persil Power Pro Original x30,Dishwash,2947365,Ecover dishwasher tablets x25,Dishwash,496,4467,0,0.000000,0.000000e+00,8,28,-0.423698
3,7614246,Persil Power Pro Lemon x30,Dishwash,2947365,Ecover dishwasher tablets x25,Dishwash,844,4467,0,0.000000,0.000000e+00,6,30,-0.450226
4,7680058,Finish Quantum Regular X18,Dishwash,2947365,Ecover dishwasher tablets x25,Dishwash,4230,6222,0,0.000000,0.000000e+00,13,30,-0.383280
5,7512242,Fairy Platinum Original x30,Dishwash,2947365,Ecover dishwasher tablets x25,Dishwash,9239,5326,0,0.000000,0.000000e+00,13,30,-0.417984
6,7513133,Fairy for Dishwashers x34,Dishwash,2947365,Ecover dishwasher tablets x25,Dishwash,7829,5548,0,0.000000,0.000000e+00,20,30,-0.377374
7,7614268,Persil Power Pro Original x15,Dishwash,2947365,Ecover dishwasher tablets x25,Dishwash,543,4467,0,0.000000,0.000000e+00,7,24,-0.454688
8,7512416,Fairy for Dishwashers Lemon x34,Dishwash,2947365,Ecover dishwasher tablets x25,Dishwash,13287,5548,0,0.000000,0.000000e+00,13,30,-0.434300
9,7512240,Fairy Platinum Lemon x30,Dishwash,2947365,Ecover dishwasher tablets x25,Dishwash,11258,5326,0,0.000000,0.000000e+00,15,30,-0.394791
10,7680127,Finish All in One Regular X56,Dishwash,2947365,Ecover dishwasher tablets x25,Dishwash,10225,4489,0,0.000000,0.000000e+00,7,30,-0.357809
11,7734378,Finish All In One S&P Regular x22,Dishwash,2947365,Ecover dishwasher tablets x25,Dishwash,3253,5648,2,0.000615,1.088557e-07,19,30,-0.391023


In [19]:
hnp

,INFERRED_CUSTOMER_ID,TRANSACTION_DATE,TRANSACTION_TIME,TRANSACTION_NUMBER,LOCATION_KEY,LOCATION,LOC_NAME,TILL_NUMBER,SKU,SKU_DESC,PARTY_ACCOUNT_TYPE_CODE,item_quantity,extended_price,UNIQUE_TRANSACTION_ID,prom_week_no,promotions,UNIQUE_CUST_TRANS
0,20000325995237,16FEB2015,145313,129,464,6,Chichester,31,310963,JS LAVENDER BEESWAX FURNTRE POLISH 300ML,4,1,0.80,2013514531312946431,201448,0,2000032599523716FEB2015
1,20000317952485,17FEB2015,113602,26,690,686,Brookwood,12,310963,JS LAVENDER BEESWAX FURNTRE POLISH 300ML,4,1,0.80,201361136022669012,201448,0,2000031795248517FEB2015
2,20000387904958,17FEB2015,194301,286,669,664,Chippenham,20,310963,JS LAVENDER BEESWAX FURNTRE POLISH 300ML,4,1,0.80,2013619430128666920,201448,0,2000038790495817FEB2015
3,20000354435104,15FEB2015,115249,44,685,680,Water Lane,13,310963,JS LAVENDER BEESWAX FURNTRE POLISH 300ML,4,1,0.80,201341152494468513,201448,0,2000035443510415FEB2015
4,20000046073708,11FEB2015,71845,45,464,6,Chichester,221,310963,JS LAVENDER BEESWAX FURNTRE POLISH 300ML,2,1,0.80,201307184545464221,201448,0,2000004607370811FEB2015
5,40000977049634,13FEB2015,65724,55,659,652,Burpham,221,310970,JS NATURAL BEESWAX FURNITRE POLISH 300ML,2,1,0.80,201326572455659221,201448,0,4000097704963413FEB2015
6,20000413494401,15FEB2015,141940,45,461,3,Pepper Hill,27,310970,JS NATURAL BEESWAX FURNITRE POLISH 300ML,4,1,0.80,201341419404546127,201448,0,2000041349440115FEB2015
7,20000246118474,15FEB2015,135725,87,517,116,Haywards Heath,25,310970,JS NATURAL BEESWAX FURNITRE POLISH 300ML,4,1,0.80,201341357258751725,201448,0,2000024611847415FEB2015
8,40000925269994,13FEB2015,113747,169,18966,2168,Stanway,221,310970,JS NATURAL BEESWAX FURNITRE POLISH 300ML,2,1,0.80,2013211374716918966221,201448,0,4000092526999413FEB2015
9,20000316670223,15FEB2015,144501,107,620,556,Hempstead Valley,17,310970,JS NATURAL BEESWAX FURNITRE POLISH 300ML,4,1,0.80,2013414450110762017,201448,0,2000031667022315FEB2015


In [40]:
SKU_A = 7512416
SKU_B = 2947365
hnp_pair = hnp[hnp['SKU'].isin([SKU_A,SKU_B])][['INFERRED_CUSTOMER_ID', 'SKU']].drop_duplicates()
sku_count_by_customer = hnp_pair.groupby( ['INFERRED_CUSTOMER_ID']).count()

In [61]:
customer_list = sku_count_by_customer[sku_count_by_customer['SKU'] == 2].index.values

In [62]:
data_pair = hnp[hnp['SKU'].isin([SKU_A,SKU_B]) & (hnp['INFERRED_CUSTOMER_ID'].isin(customer_list))]

In [63]:
data_pair

,INFERRED_CUSTOMER_ID,TRANSACTION_DATE,TRANSACTION_TIME,TRANSACTION_NUMBER,LOCATION_KEY,LOCATION,LOC_NAME,TILL_NUMBER,SKU,SKU_DESC,PARTY_ACCOUNT_TYPE_CODE,item_quantity,extended_price,UNIQUE_TRANSACTION_ID,prom_week_no,promotions,UNIQUE_CUST_TRANS
16256,40000994863855,16FEB2015,73418,61,560,415,Springfield,221,7512416,Fairy for Dishwashers Lemon x34,2,2,12.0,201357341861560221,201448,1,4000099486385516FEB2015
28546,20000396284076,22FEB2015,81404,176,688,683,Kiln Lane,222,2947365,Ecover dishwasher tablets x25,2,1,3.5,2014181404176688222,201449,1,2000039628407622FEB2015
79510,40000994863855,27FEB2015,73442,37,560,415,Springfield,222,7512416,Fairy for Dishwashers Lemon x34,2,1,6.0,201467344237560222,201450,1,4000099486385527FEB2015
79602,20000441254752,25FEB2015,112850,88,671,666,Badger Farm,14,7512416,Fairy for Dishwashers Lemon x34,4,1,6.0,201441128508867114,201450,1,2000044125475225FEB2015
94745,20000024363401,09MAR2015,103757,205,471,17,Crayford,221,2947365,Ecover dishwasher tablets x25,2,1,5.0,20156103757205471221,201451,0,2000002436340109MAR2015
111921,40000994863855,10MAR2015,72805,47,560,415,Springfield,222,7512416,Fairy for Dishwashers Lemon x34,2,1,8.0,201577280547560222,201451,0,4000099486385510MAR2015
175003,40000994863855,20MAR2015,114626,138,560,415,Springfield,222,7512416,Fairy for Dishwashers Lemon x34,2,1,8.0,20167114626138560222,201501,0,4000099486385520MAR2015
189488,40000945734005,26MAR2015,131348,46,663,657,Hedge End,40,2947365,Ecover dishwasher tablets x25,4,1,5.0,201731313484666340,201502,0,4000094573400526MAR2015
235107,20000383759997,02APR2015,130229,36,517,116,Haywards Heath,27,7512416,Fairy for Dishwashers Lemon x34,4,1,6.0,201801302293651727,201503,1,2000038375999702APR2015
316513,20000441254752,23APR2015,120411,14,671,666,Badger Farm,21,2947365,Ecover dishwasher tablets x25,4,1,3.5,202011204111467121,201506,1,2000044125475223APR2015


In [78]:
SKU_list = [7512240, 2947365, 7495079, 7512244, 7512320, 7512370, 7512427, 7624019]
txn_test = txn_data[txn_data['SKU'].isin(SKU_list)]

In [81]:
def classify_sisters(row):
    if row['SKU'] == 7512240:
        return 'Parent'
    else:
        return 'Sister'

txn_test['Status'] = txn_test.apply(classify_sisters, axis = 1)

C:\Users\Anurag.Gandhi\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [82]:
txn_test

,INFERRED_CUSTOMER_ID,TRANSACTION_DATE,TRANSACTION_TIME,TRANSACTION_NUMBER,LOCATION_KEY,LOCATION,LOC_NAME,TILL_NUMBER,SKU,SKU_DESC,...,extended_price,UNIQUE_TRANSACTION_ID,NEW_DATE,CALENDAR_DATE,PROM_WEEK_NO,Promo_Cycle,promotions,promotions_NC,UNIQUE_CUST_TRANS,Status
5,20000000007910,27FEB2014,133451,75,716,727,Fosse Park,16,7512240,Fairy Platinum Lemon x30,...,5.0,197811334517571616,2014-02-27,02/27/2014,201350,2013_17,1,0,2000000000791027FEB2014,Parent
56,20000000007910,29MAY2014,172751,148,716,727,Fosse Park,18,7624019,Fairy Platinum Tablets Lemon x55,...,9.0,1987217275114871618,2014-05-29,05/29/2014,201411,2014_04,1,0,2000000000791029MAY2014,Sister
71,20000000007910,27JUN2014,94634,25,716,727,Fosse Park,16,7624019,Fairy Platinum Tablets Lemon x55,...,9.0,19901946342571616,2014-06-27,06/27/2014,201415,2014_05,1,0,2000000000791027JUN2014,Sister
99,20000000007910,28AUG2014,161309,99,716,727,Fosse Park,16,7512240,Fairy Platinum Lemon x30,...,6.0,199631613099971616,2014-08-28,08/28/2014,201424,2014_09,1,0,2000000000791028AUG2014,Parent
125,20000000007910,06NOV2014,182855,137,716,727,Fosse Park,16,7624019,Fairy Platinum Tablets Lemon x55,...,10.0,2003318285513771616,2014-11-06,11/06/2014,201434,2014_12,1,0,2000000000791006NOV2014,Sister
240,20000000007910,03OCT2015,92526,20,716,727,Fosse Park,19,7624019,Fairy Platinum Tablets Lemon x55,...,10.0,20364925262071619,2015-10-03,10/03/2015,201529,2015_10,1,0,2000000000791003OCT2015,Sister
301,20000000017011,04MAY2014,154514,130,685,680,Water Lane,30,7512240,Fairy Platinum Lemon x30,...,5.0,1984715451413068530,2014-05-04,05/04/2014,201407,2014_03,1,0,2000000001701104MAY2014,Parent
318,20000000017011,18JUL2014,150112,144,685,680,Water Lane,26,7512240,Fairy Platinum Lemon x30,...,5.0,1992215011214468526,2014-07-18,07/18/2014,201418,2014_06,1,0,2000000001701118JUL2014,Parent
458,20000000017011,09FEB2016,165656,47,685,680,Water Lane,25,7512244,Fairy Platinum Lemon x20,...,5.0,204931656564768525,2016-02-09,02/09/2016,201547,2015_17,0,0,2000000001701109FEB2016,Sister
505,20000000037807,06AUG2014,174149,143,579,503,Leamington,18,7512244,Fairy Platinum Lemon x20,...,4.0,1994117414914357918,2014-08-06,08/06/2014,201421,2014_07,1,0,2000000003780706AUG2014,Sister


In [84]:
txn_test.to_csv('boundary_transactions.csv')

In [88]:
txn_data[(txn_data['INFERRED_CUSTOMER_ID'] == 40000977860936) & (txn_data['SKU'] == 7512242)]


,INFERRED_CUSTOMER_ID,TRANSACTION_DATE,TRANSACTION_TIME,TRANSACTION_NUMBER,LOCATION_KEY,LOCATION,LOC_NAME,TILL_NUMBER,SKU,SKU_DESC,...,item_quantity,extended_price,UNIQUE_TRANSACTION_ID,NEW_DATE,CALENDAR_DATE,PROM_WEEK_NO,Promo_Cycle,promotions,promotions_NC,UNIQUE_CUST_TRANS
10929178,40000977860936,08JAN2016,54708,4,509,63,Sevenoaks,222,7512242,Fairy Platinum Original x30,...,1,6,20461547084509222,2016-01-08,01/08/2016,201543,2015_15,1,0,4000097786093608JAN2016


In [89]:
sisters_list[(sisters_list['SKU_1'] == 7512240) & (sisters_list['SKU_2'] == 7512242)]

,index,buyer_id_2,buyer_resp_2,sub_category_2,sub_category_name_2,segment_2,segment_name_2,sku_desc_2,buyer_id_1,buyer_resp_1,...,SKU_2,count_stores,avg_corr,avg_corr_14,avg_corr_np,SUPPORT A,SUPPORT B,SUPPORT A_B,CONFIDENCE,LIFT
